In [1]:
import lasagne
import theano
import lasagne.layers as L
import theano.tensor as T

In [22]:
# self._init_model(in_size, out_size, n_hid=n_hid, learning_rate_sl=lr, batch_size=batch, \
#                 ment=ment)
in_size = 24
out_size = 7
n_hid = 23
lr = .01
batch = 128
ment = 0

In [95]:
input_var, turn_mask, act_mask, reward_var = T.dtensor3('in'), T.imatrix('tm'), T.itensor3('am'), T.dvector('r')
pol_in = T.dmatrix('pol-h')

'''shape:tuple of int or None elements
The shape of the input. Any element can be None to indicate that the size of that dimension is not fixed at compile time.

input_var:Theano symbolic variable or None (default: None)
A variable representing a network input. If it is not provided, a variable will be created.'''

l_mask_in = L.InputLayer(shape=(None,None), input_var=turn_mask)

l_in = L.InputLayer(shape=(None, None, in_size), input_var=input_var)


l_pol_rnn = L.GRULayer(l_in, 
                       n_hid, 
                       hid_init=pol_in, 
                       mask_input=l_mask_in) # B x H x D

pol_out = L.get_output(l_pol_rnn)[:,-1,:]
l_den_in = L.ReshapeLayer(l_pol_rnn, (turn_mask.shape[0] * turn_mask.shape[1], n_hid)) # BH x D
l_out = L.DenseLayer(l_den_in, out_size, nonlinearity=lasagne.nonlinearities.softmax)

network = pol_out
probs = L.get_output(l_out) # BH x A
out_probs = T.reshape(probs, (input_var.shape[0], input_var.shape[1], out_size)) # B x H x A


log_probs = T.log(out_probs)

act_probs = (log_probs * act_mask).sum(axis=2) # B x H
ep_probs = (act_probs * turn_mask).sum(axis=1) # B
H_probs = -T.sum(T.sum(out_probs * log_probs, axis=2), axis=1) # B
loss = 0. - T.mean(ep_probs * reward_var + ment * H_probs)

In [100]:
act_fn = theano.function([input_var, turn_mask, pol_in], [out_probs, log_probs])

F:\Anaconda3\envs\torch_kbinfo\lib\site-packages\theano\scan_module\scan_perform_ext.py:76: UserWarning: The file scan_perform.c is not available. This donot happen normally. You are probably in a strangesetup. This mean Theano can not use the cython code for scan. If youwant to remove this warning, use the Theano flag'cxx=' (set to an empty string) to disable all ccode generation.
  "The file scan_perform.c is not available. This do"


In [70]:
import numpy as np

In [71]:
i = np.random.random((2, 10, 24))
t = np.array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])
a = np.array([[[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]],

       [[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]])

r = np.array([-1.09, -1.09])

pi = np.zeros((2, n_hid)).astype('float32')

In [124]:
np.repeat(np.arange(5), 2, axis=0)

array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4])

In [137]:
z = np.arange(5)
np.repeat(z[np.newaxis,:], 2, axis=0)

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [ ]:
np.zeros()

In [134]:
z[np.newaxis,:]

array([[0, 1, 2, 3, 4]])

In [72]:
t.shape

(2, 10)

In [101]:
p, lp = act_fn(i, t, pi)

In [103]:
p.shape, lp.shape

((2, 10, 7), (2, 10, 7))

In [104]:
a.shape

(2, 10, 7)

In [119]:
-((p * lp).sum(axis=2)).sum(axis=1)

array([19.29326408, 19.32539854])

In [106]:
a

array([[[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]],

       [[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]])

In [141]:
t.shape

(2, 10)

In [140]:
a.shape

(2, 10, 7)

In [139]:
(lp * a).sum(axis=2).shape

(2, 10)

In [ ]:
пу

In [121]:
act_probs = (lp * a).sum(axis=2)
((act_probs * t).sum(axis=1))

array([-3.99131469, -3.87455499])

In [112]:
t

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [99]:
lp

array([[[-1.89332186, -2.10473527, -2.0193182 , -1.81631342,
         -1.9813618 , -1.96364102, -1.87110583],
        [-2.05852003, -2.18022749, -2.25274145, -1.87813225,
         -1.82148052, -1.67142021, -1.88657941],
        [-2.05852003, -2.18022749, -2.25274145, -1.87813225,
         -1.82148052, -1.67142021, -1.88657941],
        [-2.05852003, -2.18022749, -2.25274145, -1.87813225,
         -1.82148052, -1.67142021, -1.88657941],
        [-2.05852003, -2.18022749, -2.25274145, -1.87813225,
         -1.82148052, -1.67142021, -1.88657941],
        [-2.05852003, -2.18022749, -2.25274145, -1.87813225,
         -1.82148052, -1.67142021, -1.88657941],
        [-2.05852003, -2.18022749, -2.25274145, -1.87813225,
         -1.82148052, -1.67142021, -1.88657941],
        [-2.05852003, -2.18022749, -2.25274145, -1.87813225,
         -1.82148052, -1.67142021, -1.88657941],
        [-2.05852003, -2.18022749, -2.25274145, -1.87813225,
         -1.82148052, -1.67142021, -1.88657941],
        [-

In [86]:
gru[0].reshape(20, 23)

array([[-0.06375682,  0.08783907,  0.11329573,  0.07477202,  0.02013782,
        -0.00679636,  0.07680428,  0.12398521, -0.09848753, -0.01654078,
         0.02089729,  0.08538025,  0.27566153, -0.10790463,  0.07739172,
         0.04850656,  0.18752919, -0.09548239, -0.07806815, -0.02427692,
        -0.16886144,  0.01422618,  0.11407929],
       [-0.20334518,  0.11476226,  0.14196033, -0.0555009 ,  0.20653439,
        -0.01830684,  0.03638172,  0.26492145, -0.0553292 , -0.17479163,
         0.03462211,  0.26032273,  0.34562005, -0.27044844,  0.13741814,
        -0.07877142,  0.30980053, -0.11555158, -0.082664  , -0.00367976,
        -0.32400075,  0.01854942,  0.05127486],
       [-0.20334518,  0.11476226,  0.14196033, -0.0555009 ,  0.20653439,
        -0.01830684,  0.03638172,  0.26492145, -0.0553292 , -0.17479163,
         0.03462211,  0.26032273,  0.34562005, -0.27044844,  0.13741814,
        -0.07877142,  0.30980053, -0.11555158, -0.082664  , -0.00367976,
        -0.32400075,  0.0185

In [56]:
gru[0].shape

(2, 10, 23)

In [46]:
inp_d = np.array(np.arange(in_size)).reshape(1, 1, in_size)
tur_d = np.ones((inp_d.shape[0], inp_d.shape[1])).astype('int8')
pol_d = np.zeros((1, n_hid))

In [47]:
act_p, p_out = act_fn(inp_d, tur_d, pol_d)

In [52]:
act_p.shape

(1, 1, 7)